In [ ]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import scipy.spatial as scp

In [ ]:
state_frame = gpd.read_file('/media/isri/T7/Region/RegionDCL/data/projected/Singapore/poi/poi.shp')

In [ ]:
df = state_frame[['fclass', 'code', 'geometry']]
subset= df.copy()
subset['latitude'] = subset['geometry'].apply(lambda x: x.y)
subset['longitude'] = subset['geometry'].apply(lambda x: x.x)
subset['name'] = subset['fclass']

In [ ]:
poi_csv = subset[['name', 'latitude', 'longitude']].copy()
poi_csv.to_csv('poi_sg.csv')

In [ ]:
name_list = []
coordinate_list = []
for i, item in subset.iterrows():
    name_list.append(item['name'])
    coordinate_list.append([item['longitude'], item['latitude']])

In [ ]:
ordered_coordinates = scp.cKDTree(coordinate_list)

In [ ]:
with open('SPABERT_finetuning_data(seoul_5).json', 'w') as out_f:
    for i, item in subset.iterrows():
        name = item['name']
        lat = item['latitude']
        lon = item['longitude']
        coordinates = [lon, lat]
        
        _, nearest_neighors_idx = ordered_coordinates.query([coordinates], k=5)
        
        nearest_name = []
        nearest_coordinates = []
        
        for idx in nearest_neighors_idx[0]:
            neighbor_name = name_list[idx]
            neighbor_coordinates = coordinate_list[idx]
            nearest_name.append(neighbor_name)
            nearest_coordinates.append({'coordinates': neighbor_coordinates})
            
        neighbor_info = {'name_list': nearest_name, 'geometry_list': nearest_coordinates}
        
        place = {'info':{'name': name, 'geometry':{'coordinates': coordinates}}, 'neighbor_info': neighbor_info}
        
        out_f.write(json.dumps(place))
        out_f.write('\n')